위 신문 데이터를 바탕으로 신문 내용별 분류기를 개발하라 <br></br>
위 데이터를 Traing / Test Dataset으로 나눠서 5-fold cross validation(5번 데이터를 training / testset으로 나눔, KV 활용)<br></br>
Naive Bayesian Classifier와 Count Vector를 활용하여 각각 성능을 테스트하라<br></br>
NB는 multinomial과 bernuoil 분포를 모두 사용하라<br></br>
가능할 경우, TF-IDF vector를 활용해 볼것 (검색어 - tf-idf scikit-learn)

In [1]:
import sklearn as sk
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
import re

In [2]:
news = fetch_20newsgroups(subset='all')

In [3]:
print(news.keys())

print(news.target_names[3])

dict_keys(['data', 'filenames', 'target_names', 'DESCR', 'target', 'description'])
comp.sys.ibm.pc.hardware


In [4]:
#분류 코드는 0부터 19까지 있다.
stemmer = SnowballStemmer('english')

x_data = news['data'] #data 
y_data = news['target']


##### 데이터 전처리

1. 문장 부호 제거
2. 숫자 제거
3. 영어 소문자화
4. 불용어 제거

In [5]:
#이메일 제거 및 단어 원형복원
def delete_email(X):
    for i in range(len(X)):
        new_list = X[i].split()
        for j in range(len(new_list)):
            new_list[j] = stemmer.stem(new_list[j])
            if '@' in new_list[j] :
                new_list[j] = ""
                #del new_list[i]
        X[i] =  " ".join(new_list)       
    return X    

#불필요한 text 제거
def delete_text(X):
    for i in range(len(X)):
        X[i] = X[i].replace("\n", "")
        X[i] = X[i].replace("\t", "")
        X[i] = X[i].replace("subject", "")
        X[i] = X[i].replace("from", "")
       
    return X

#특수부호 제거
def delete_(X):
    for i in range(len(X)):
        X[i] =re.sub('[-=#/)^_&12?345+(67890|:$}!,{.><*]', '', X[i])
    return X


x_data = delete_email(x_data)
x_data = delete_text(x_data)
x_data = delete_(x_data)

print(x_data[:2])

[' mamatha devineni ratnam   pen fan reaction organization post office carnegi mellon pittsburgh pa lines  nntppostinghost poandrewcmuedu i am sure some basher of pen fan are pretti confus about the lack of ani kind of post about the recent pen massacr of the devils actually i am bit puzzl too and a bit relieved however i am go to put an end to nonpittsburgh relief with a bit of prais for the pens man they are kill those devil wors than i thought jagr just show you whi he is much better than his regular season stats he is also a lot fo fun to watch in the playoffs bowman should let jagr have a lot of fun in the next coupl of game sinc the pen are go to beat the pulp out of jersey anyway i was veri disappoint not to see the island lose the final regular season game pen rule', '  matthew b lawson  which highperform vlb video card summary seek recommend for vlb video card nntppostinghost midwayecnuoknoredu organization engin comput network univers of oklahoma norman ok usa keywords orchid

In [6]:
#stop_words 불용어 처리
vectorizer = TfidfVectorizer(stop_words='english')

tfidf_matrix = vectorizer.fit_transform(x_data)

#set data
X = tfidf_matrix

y  = y_data

In [7]:
clf_M = MultinomialNB()
clf_M.fit(X, y)

clf_B = BernoulliNB()
clf_B.fit(X, y)


#5-fold cross validation
scores_M = cross_val_score(clf_M, X, y, cv=5)
print("Multinomial Accuracy: %0.2f (+/- %0.2f)" % (scores_M.mean(), scores_M.std() * 2))

scores_B = cross_val_score(clf_B, X, y, cv=5)
print("Bernoulli Accuracy: %0.2f (+/- %0.2f)" % (scores_B.mean(), scores_B.std() * 2))




Multinomial Accuracy: 0.87 (+/- 0.01)
Bernoulli Accuracy: 0.71 (+/- 0.02)
